# Import geo-data of "ÖV" in Geneva using swisstopo API

## Libraries and settings

In [35]:
# Libraries
import os
import requests
import json
import folium
from pandas import json_normalize

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Show current working directory
print(os.getcwd())

/workspaces/project_arm_airbnb/data_oev


## API request

### Train stations

In [36]:
# Overpass API URL
url = 'http://overpass-api.de/api/interpreter'

# Overpass turbo query
query = f"""
        [out:json];
        area["ISO3166-2"="CH-GE"][admin_level=4];
        node ["railway"="station"](area);
        out;"""

# Web API request
r = requests.get(url, params={'data': query})
data = r.json()['elements']

# Save data to file
with open('station.json', 'w') as json_file:
    json.dump(data, json_file)

# Store data in data frame
df = json_normalize(data)

# Number of rows and columns
print(df.shape)

# First rows
df.head()

(13, 38)


,type,id,lat,lon,tags.ele,tags.internet_access,tags.internet_access:fee,tags.internet_access:operator,tags.name,tags.name:de,...,tags.name:azb,tags.name:en,tags.name:fa,tags.name:kk-Arab,tags.name:pnb,tags.name:sd,tags.name:ur,tags.network,tags.network:short,tags.network:wikidata
0,node,2388447150,46.231334,6.110271,424,wlan,no,SBB,Genève-Aéroport,Genf Flughafen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,2754246079,46.220426,6.094808,428,NaN,NaN,NaN,Vernier,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node,3053167578,46.263684,6.161146,386,NaN,NaN,NaN,Creux-de-Genthod,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node,3053167580,46.279740,6.165805,388,no,NaN,NaN,Versoix,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node,3057742963,46.222273,6.076288,434,NaN,NaN,NaN,Meyrin,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Tram stops

In [37]:
# Overpass turbo query
query = f"""
        [out:json];
        area["ISO3166-2"="CH-GE"][admin_level=4];
        node ["railway"="tram_stop"](area);
        out;"""

# Web API request
r = requests.get(url, params={'data': query})
data = r.json()['elements']

# Save data to file
with open('tram.json', 'w') as json_file:
    json.dump(data, json_file)

# Store data in data frame
df = json_normalize(data)

# Number of rows and columns
print(df.shape)

# First rows
df.head()


(180, 29)


,type,id,lat,lon,tags.ele,tags.name,tags.network,tags.operator,tags.public_transport,tags.railway,...,tags.website,tags.url,tags.contact:website,tags.layer,tags.opening_hours,tags.addr:city,tags.addr:postcode,tags.addr:street,tags.note,tags.start_date
0,node,35416998,46.218462,6.110622,430,Balexert,18,TPG,stop_position,tram_stop,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,35420048,46.221020,6.141444,401,Collège Sismondi,NaN,TPG,stop_position,tram_stop,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node,36663771,46.188926,6.131024,378,Pictet-Thellusson,NaN,TPG,stop_position,tram_stop,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node,297910640,46.200564,6.140755,375,Cirque,NaN,TPG,stop_position,tram_stop,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node,297914745,46.200847,6.143642,382,Place de Neuve,Unireso,TPG,stop_position,tram_stop,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Bus stops

In [38]:
# Overpass turbo query
query = f"""
        [out:json];
        area["ISO3166-2"="CH-GE"][admin_level=4];
        node ["highway"="bus_stop"](area);
        out;"""

# Web API request
r = requests.get(url, params={'data': query})
data = r.json()['elements']

# Save data to file
with open('bus.json', 'w') as json_file:
    json.dump(data, json_file)

# Store data in data frame
df = json_normalize(data)

# Number of rows and columns
print(df.shape)

# First rows
df.head()

(1451, 54)


,type,id,lat,lon,tags.bus,tags.ele,tags.highway,tags.name,tags.operator,tags.public_transport,...,tags.contact:website,tags.name:zh,tags.nat_name,tags.description,tags.was:highway,tags.note,tags.website,tags.not:network:wikidata,tags.start_date,tags.tourist_bus
0,node,497307,46.222755,6.139638,yes,372,bus_stop,Nations,TPG,stop_position,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,35419951,46.220048,6.145630,yes,390,bus_stop,Maison de la Paix,TPG,stop_position,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node,36473383,46.156441,6.003172,yes,422,bus_stop,Athenaz-Passeiry,TPG,stop_position,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node,36473725,46.144418,5.965679,yes,341,bus_stop,Chancy-Douane,TPG,stop_position,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node,36647210,46.220019,6.085083,yes,NaN,bus_stop,Signal,TPG,stop_position,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Airport

In [49]:
# Overpass turbo query
query = f"""
        [out:json];
        area["ISO3166-1"="CH"][admin_level=2];
        node ["aeroway"="aerodrome"](area);
        out;"""

# Web API request
r = requests.get(url, params={'data': query})
data = r.json()['elements']

# Save data to file
with open('airports.json', 'w') as json_file:
    json.dump(data, json_file)

# Store data in data frame
df = json_normalize(data)

# Number of rows and columns
print(df.shape)

# First rows
df.head()

(17, 29)


,type,id,lat,lon,tags.aerodrome,tags.aerodrome:type,tags.aeroway,tags.alt_name,tags.ele,tags.iata,...,tags.name:en,tags.operator,tags.source,tags.addr:city,tags.addr:housenumber,tags.addr:postcode,tags.addr:street,tags.email,tags.phone,tags.internet_access
0,node,60647148,47.485960,9.559860,regional,private,aerodrome,Business Airport St.Gallen - Altenrhein,398,ACH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,256570718,47.689802,8.527235,NaN,NaN,aerodrome,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node,289344453,47.183141,7.742184,regional,NaN,aerodrome,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node,364736401,46.594942,7.094254,NaN,NaN,aerodrome,NaN,NaN,NaN,...,La Gruyère Airfield,Société d’Aviation de la Gruyère S.A.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node,1042047734,46.297793,8.991557,NaN,NaN,aerodrome,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
